In [ ]:
# Common imports
%matplotlib inline
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns  #need to conda install seaborn
import sys
getsize=sys.getsizeof
#pd.set_option('max_columns', 50)

In [ ]:
# Pandas Series and Dataframes

import pandas as pd
# pandas Series example
s = pd.Series([7, 'Heisenberg', 3.14, -1789710578, 'Happy Eating!'],
              index=['A', 'Z', 'C', 'Y', 'E'])
print('\nSERIES WITH INDEX\n' + str(s))
d = {'Chicago': 1000, 'New York': 1300, 'Portland': 900, 'San Francisco': 1100,
     'Austin': 450, 'Boston': None}
cities = pd.Series(d)
print('\nSERIES FROM DICT\n' + str(cities))

# pandas Dataframe examples
data = {'year': [2010, 2011, 2012, 2011, 2012, 2010, 2011, 2012],
        'team': ['Bears', 'Bears', 'Bears', 'Packers', 'Packers', 'Lions', 'Lions', 'Lions'],
        'wins': [11, 8, 10, 15, 11, 6, 10, 4],
        'losses': [5, 8, 6, 1, 5, 10, 6, 12]}
football = pd.DataFrame(data, columns=['year', 'team', 'wins', 'losses'])
football.head()
football.to_excel('data/football.xlsx', index=False)
football1 = pd.read_excel('data/football.xlsx', 'Sheet1')
football1

plt.rcParams["figure.figsize"] = (10,7)
football.wins.plot.bar()  # plot options: area, bar, barh, box, density, hist, kde, line, pie
plt.title("plot of football wins")
plt.ylabel('number of wins')
plt.xlabel('wins')

# graphical filtering of dataframe 
import qgrid
qgrid.nbinstall(overwrite=True)
qgrid.set_defaults(remote_js=True)
#need to click export for some reason to get to appear
qgrid.show_grid(football, show_toolbar=True)

In [ ]:
# pandas loading a dataframe

from IPython.display import IFrame
import pandas as pd
# READING FROM A TEXT FILE
#from_csv = pd.read_csv('pandas_data.csv')
#from_csv.head()

# READING FROM A SQL DATABASE
from sqlalchemy import create_engine
from pandas.io import sql

db = create_engine('sqlite:///data/sample1.db')
conn = db.connect()
#creation = conn.execute("Create table btable (a varchar(10))")
insertion = conn.execute("Insert into btable values ('hello')" )
query = 'SELECT * from btable'
selection = conn.execute(query)
#for row in selection:
#    print("AAAA:", row['a'])
df1 = sql.read_sql(query, con=conn)
print(df1.head())

# READ FROM THE CLIPBOARD
#df = pd.read_clipboard()
#df.head()

#READ FROM URL
url = 'https://raw.github.com/gjreda/best-sandwiches/master/data/best-sandwiches-geocode.tsv'
from_url = pd.read_table(url, sep='\t')
#from_url.head()

#IFrame('http://www.eia.gov/coal/data.cfm',width=700,height=350)
IFrame('https://www.vocalink.com/media/1518/valacdos.txt',width=700,height=350)

In [ ]:
# pandas data manipulation

heading = ['classify', 'message']
sms = pd.read_table('data/sms.txt', header=None, names = heading)
print('info:\n',sms.info(),'\n')
print('dtypes:\n',sms.dtypes,'\n')
#print(sms.head(),'\n')
print('slice:\n',sms[:2],'\n')
print('filter:\n',sms[(sms.classify == 'spam') & (sms.message != 'qwe')].head(),'\n')
print('loc:\n',sms.loc[100],'\n')
#print('describe:\n',sms.describe,'\n')

sms1 = sms.copy()
newsms = pd.merge(sms, sms1, on='classify', how='inner')  # equivalent of SQL inner join, 
# outer joins are how='left', how='right', how='outer
newsms1 = pd.concat([sms, sms1])
newsms2 = pd.concat([sms, sms1], axis=1)

#grouping data like with SQL groupby
newsms3 = sms.groupby('classify')
pd.unique(sms.classify)

print('count:\n',sms.count(),'\n')
print('size:\n',sms.size,'\n')
#newsms.sum()
#newsms.mean()
#newsms.median()
print('groupby:\n',sms.groupby('classify').size().sort_values(ascending=False)[:25],'\n')
print('value_counts\n',sms.classify.value_counts()[:25],'\n')

In [1]:
# get lots of data in parallel using multiprocessing (DOSENT WORK ON WINDOWS)

import requests
from multiprocessing import Process, Manager
import json
import pandas as pd

def get_story(story_id, stories):
    url = 'https://hacker-news.firebaseio.com/v0/item/%d.json' % story_id
    resp = requests.get(url)
    stories.append(resp.json())
    
def get_top_stories():
    manager = Manager()
    stories = manager.list()
    url = 'https://hacker-news.firebaseio.com/v0/topstories.json'
    ids = requests.get(url)
    processes = [Process(target=get_story, args=(sid, stories))
                 for sid in ids.json()]
    for p in processes:
        p.start()
    for p in processes:
        p.join()
    return stories

#top_stories = get_top_stories()
#list(top_stories)

In [ ]:
# train and test split with Pandas

data = pd.DataFrame(np.random.randn(100, 2))
train=data.sample(frac=0.6,random_state=20)

temp=data.drop(train.index)
test=temp.sample(frac=0.5,random_state=20)
csv=temp.drop(test.index)
del temp

len(train.index), len(test.index), len(csv.index)

In [ ]:
# Column-wise correlation to support Item collaberative filtering

import pandas as pd
import numpy as np

r_cols = ['user_id', 'movie_id', 'rating']
ratings = pd.read_csv('ml-100k/u.data', sep='\t', names=r_cols, usecols=range(3), encoding="ISO-8859-1")
m_cols = ['movie_id', 'title']
movies = pd.read_csv('ml-100k/u.item', sep='|', names=m_cols, usecols=range(2), encoding="ISO-8859-1")
ratings = pd.merge(movies, ratings)

userRatings = ratings.pivot_table(index=['user_id'],columns=['title'],values='rating')
#userRatings.head()

corrMatrix = userRatings.corr(method='pearson', min_periods=100)
#corrMatrix.head()

#Now let's produce some movie recommendations for user ID 0, who I manually added to the data set as a test case. 
#This guy really likes Star Wars and The Empire Strikes Back, but hated Gone with the Wind. I'll extract his ratings 
#from the userRatings DataFrame, and use dropna() to get rid of missing data (leaving me only with a Series of the movies 
#I actually rated:)
myRatings = userRatings.loc[0].dropna()
myRatings

#Now, let's go through each movie I rated one at a time, and build up a list of possible recommendations based on the 
#movies similar to the ones I rated. So for each movie I rated, I'll retrieve the list of similar movies from our 
#correlation matrix. I'll then scale those correlation scores by how well I rated the movie they are similar to, so movies 
#similar to ones I liked count more than movies similar to ones I hated:
simCandidates = pd.Series()
for i in range(0, len(myRatings.index)):
    print ("Adding sims for " + myRatings.index[i] + "...")
    # Retrieve similar movies to this one that I rated
    sims = corrMatrix[myRatings.index[i]].dropna()
    # Now scale its similarity by how well I rated this movie
    sims = sims.map(lambda x: x * myRatings[i])
    # Add the score to the list of similarity candidates
    simCandidates = simCandidates.append(sims)
    
#Glance at our results so far:
print ("sorting...")
simCandidates.sort_values(inplace = True, ascending = False)
print (simCandidates.head(10))

#This is starting to look like something useful! Note that some of the same movies came up more than once, 
#because they were similar to more than one movie I rated. We'll use groupby() to add together the scores from movies 
#that show up more than once, so they'll count more:
simCandidates = simCandidates.groupby(simCandidates.index).sum()

simCandidates.sort_values(inplace = True, ascending = False)
simCandidates.head(10)
#The last thing we have to do is filter out movies I've already rated, 
filteredSims = simCandidates.drop(myRatings.index)
filteredSims.head(10)

In [ ]:
# matplotlib plotting, pie chart, bar chart

from math import log, sin, pi
from scipy.special import expit
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn-notebook')  #plt.style.use('bmh')
plt.rcParams.update({'font.size': 20, 'font.family': 'sans-serif'})

# Set up runtime comparisons
n = np.linspace(1,10,1000)
labels = ['Constant','Logarithmic','Linear','Log Linear','Quadratic','Cubic','Exponential','Sine', 'Sigmoid']
big_o = [np.ones(n.shape), np.log(n), n, n*np.log(n), n**2, n**3, 2**n,  np.sin(n*4)*10+10, expit(n-5)*20]
# Plot setup
plt.figure(figsize=(12,8))
plt.ylim(0,50)
for i in range(len(big_o)):
    plt.plot(n,big_o[i],label = labels[i], linewidth=3)
plt.legend(loc='best')
plt.ylabel(r'$y = \alpha$', fontsize=20)
plt.xlabel(r'$x = \beta$', fontsize=20)
plt.grid(True)
plt.annotate('An Interesting Point', xy=(5, 20), arrowprops=dict(arrowstyle='->'), xytext=(3.1, 25))
plt.show()
# plt.savefig('C:\\Users\\Graham\Desktop\\qwe.jpg')

values = [12, 55, 4, 32, 14]
colors = ['r', 'g', 'b', 'c', 'm']
explode = [0, 0, 0.1, 0, 0.1]
labels = ['India', 'United States', 'Russia', 'China', 'Europe']
plt.figure(figsize=(12,8))
plt.pie(values, colors= colors, labels=labels, explode = explode)
plt.title('Student Locations')
plt.show()

values = [12, 55, 4, 32, 14]
colors = ['r', 'g', 'b', 'c', 'm']
plt.yscale("log")
plt.bar(range(0,5), values, color= colors)
plt.show()

uniformSkewed = np.random.rand(100) * 100 - 40
high_outliers = np.random.rand(10) * 50 + 100
low_outliers = np.random.rand(10) * -50 - 100
data = np.concatenate((uniformSkewed, high_outliers, low_outliers))
plt.boxplot(data)
plt.show()

In [ ]:
# scatter plotting to find correlations

from pandas.tools.plotting import scatter_matrix
import seaborn as sns
import pandas as pd

df1=pd.DataFrame(np.random.randn(200,4), columns=['C1', 'C2', 'C3','C4'])
df1['C5'] = df1['C4'] * df1['C3'] * 10
df1['C6'] = df1['C3'].div(3).round(decimals=1)
df1.plot.scatter('C1','C5',figsize=(5,4))
sns.pairplot(df1[['C1','C2','C3','C6']], hue='C6') #investigate correlation between pairs
scatter_matrix(df1, alpha=0.2, figsize=(8, 8), diagonal='kde')
df1.describe()

In [ ]:
# slider input of parameters

import matplotlib.pyplot as plt 
from ipywidgets import interact
%matplotlib inline
#plt.rcParams["figure.figsize"] = (6,6)

@interact
def echo(split=[0,100]):
    plt.pie([split/100, 1-split/100], labels=['this split','that split'])
    plt.show()

In [ ]:
# general plotting tinkering

%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(1)
price = pd.Series(np.random.randn(155).cumsum()+1, index=pd.date_range('2001-01-01', periods=155, freq='B'))
mavg20 = price.rolling(20).mean()
mstd20 = price.rolling(20).std()
mavg5 = price.rolling(5).mean()

plt.figure(figsize=(16,8)); plt.grid(True)
plt.plot(price.index, price, c='k')
plt.plot(mavg20.index, mavg20, c='b')
plt.plot(mavg5.index, mavg5, c='r')
plt.fill_between(mstd20.index, mavg20-2*mstd20, mavg20+2*mstd20, color='b', alpha=0.2)

In [ ]:
# correlation and covariance

#np.random.seed(10)
size=100000
X = np.matrix(np.random.randn(size))
Y = np.matrix(np.random.randn(size))
X = X + Y * 0.11 # intrroduce random noise
Xv = X - X.mean()
Yv = Y - Y.mean()

result = Xv * Yv.T
cov = float(result[[0]]) / size
corr = cov / X.std() / Y.std()
print(corr, cov)

np.corrcoef(X, Y)[0][1]

In [ ]:
#storing dataframes in CSV and HDF

%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
df1 = pd.DataFrame(pd.date_range(pd.datetime(2016, 1, 1), pd.datetime(2016, 1, 10)))
df1.columns=['date']
df1.to_csv('data/store.csv')

store = pd.HDFStore('data/store.h5')
store['df1'] =df1
store.flush
store['df1']

In [ ]:
# panel data structure, pandas IO demo, cutting dataframes

%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas.io.data as pdio
pd.options.display.float_format = '{:,.3f}'.format
start = pd.to_datetime('2016-01-01')
end = pd.to_datetime('2016-01-31')
tickers = ['MSI', 'ORCL']
msi = pdio.get_data_yahoo(tickers[0], start=start, end=end)
pan = pdio.get_data_yahoo(tickers, start=start, end=end)
pan

In [ ]:
pan.minor_xs('MSI')

In [ ]:
pan.major_xs('2016-01-04')

In [ ]:
df = pan.to_frame()
df.reset_index(inplace=True)
df

In [ ]:
df.rename(columns={'minor': 'Ticker'}, inplace=True)
df['Day'] = df.Date.dt.weekday_name
df['DayN'] = df.Date.dt.weekday
ranges = [0, 1000000, 10000000, 100000000]
labels = ['Low', 'Medium', 'High']
df['Turnover'] = pd.cut(df.Volume, bins=ranges, labels=labels)
mapper = {'MSI': 'Motorola', 'ORCL': 'Oracle'}
df.insert(2, 'Name', df.Ticker.map(mapper) )
df

In [ ]:
pd.options.display.float_format = '{:,.3f}'.format
df1=df.groupby(['Name', 'DayN', 'Day']).mean().reset_index().drop('DayN', axis=1)
df1